# Watchlist example notebook

`Author: Francisco Förster, Last update: 20210408`

Here we will do a xmatch against a list of positions in the sky, given in the file ```watchlist.csv```

In [28]:
import pandas as pd
import psycopg2
import requests

# Get credentials to open a direct connection with the database

In [29]:
credentials_file = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
params = requests.get(credentials_file).json()["params"]

In [19]:
conn = psycopg2.connect(dbname=params["dbname"], user=params["user"], host=params["host"], password=params["password"])

# Get watchlist of object positions

In [30]:
df = pd.read_csv("../example_data/watchlist.csv")
df.head()

,id_source,ra,dec
0,source_1,160.183014,33.016467
1,source_2,174.215249,44.837895


# Prepare the query string for watchlist for object within 1 degress of given positions, and  happening within the last 10 days

In [21]:
objects = []
for _,row in df.iterrows():
    objects.append(f"(\'{row.id_source}\', {row.ra}, {row.dec})")
objects_str = ",\n".join(objects)
objects_str

"('source_1', 160.18301441363647, 33.0164673528409),\n('source_2', 174.21524897555543, 44.83789535222221)"

Create query string

In [22]:
from astropy.time import Time
nt = Time.now()
last_mjd_discovery = nt.mjd - 10

In [23]:
search_radius = 1 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
)
SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd

FROM object o, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
""" % (last_mjd_discovery)
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

In [24]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(161, 8)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd
0,source_1,160.183014,33.016467,ZTF21aaqypmo,159.991994,32.595622,0.450431,59303.259306
1,source_1,160.183014,33.016467,ZTF21aaqypmp,160.168464,32.870279,0.146698,59303.259306
2,source_1,160.183014,33.016467,ZTF21aaqypum,160.632521,33.691913,0.772786,59303.259306
3,source_1,160.183014,33.016467,ZTF21aaqzdqj,160.607393,32.423670,0.692014,59303.323958
4,source_1,160.183014,33.016467,ZTF21aaqzdrw,159.517743,32.885922,0.573311,59303.323958
...,...,...,...,...,...,...,...,...
156,source_1,160.183014,33.016467,ZTF21aatjqff,159.953738,32.787785,0.298917,59312.290046
157,source_1,160.183014,33.016467,ZTF21aatjqfg,160.203328,32.539148,0.477625,59312.290046
158,source_1,160.183014,33.016467,ZTF21aatjqjd,160.337303,32.131329,0.894636,59312.290046
159,source_1,160.183014,33.016467,ZTF21aatjrey,160.700577,32.389864,0.763089,59312.291458


# Query objects within 10 degress of given positions, first detected within the last 10 days, with SN probabilities > 0.4, and ranking=1 in the stamp classifier

In [25]:
search_radius = 10 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
),
sn (oid, classifier_name, class_name, probability, ranking) AS (
    SELECT
        o.oid, p.classifier_name, p.class_name, p.probability, p.ranking
    FROM
        probability p
    INNER JOIN 
        object o
    ON 
        o.oid=p.oid
    WHERE
        p.classifier_name='stamp_classifier'
        AND p.class_name IN ('SN')
        AND p.ranking=1
        AND p.probability > 0.4
        AND o.firstmjd > {last_mjd_discovery}
)

SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd, sn.classifier_name, sn.class_name, sn.probability, sn.ranking

FROM object o INNER JOIN sn ON sn.oid=o.oid, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > {last_mjd_discovery}
ORDER BY
    c.source_id
"""
query_str = query.format(values = objects_str, radius = search_radius, last_mjd_discovery=last_mjd_discovery) # radius in degrees

In [26]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches.head(100)

(29, 12)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd,classifier_name,class_name,probability,ranking
0,source_1,160.183014,33.016467,ZTF21aasvhwh,166.770664,32.800095,5.533928,59306.372836,stamp_classifier,SN,0.707814,1
1,source_1,160.183014,33.016467,ZTF21aaroydq,163.573420,34.238864,3.076095,59306.304271,stamp_classifier,SN,0.524896,1
2,source_1,160.183014,33.016467,ZTF21aarnbox,154.133948,39.014267,7.736231,59306.303229,stamp_classifier,SN,0.431122,1
3,source_1,160.183014,33.016467,ZTF21aaqyqwk,157.083200,39.806021,7.231988,59303.273889,stamp_classifier,SN,0.755601,1
4,source_1,160.183014,33.016467,ZTF21aastzsn,148.786838,35.379682,9.711187,59310.249132,stamp_classifier,SN,0.793107,1
5,source_1,160.183014,33.016467,ZTF21aaqzdut,157.638564,39.106197,6.426903,59303.273889,stamp_classifier,SN,0.818674,1
6,source_1,160.183014,33.016467,ZTF21aardzpu,158.150713,38.960263,6.166557,59308.225984,stamp_classifier,SN,0.435725,1
7,source_1,160.183014,33.016467,ZTF21aarowtc,161.390183,39.823360,6.875642,59306.248542,stamp_classifier,SN,0.570181,1
8,source_1,160.183014,33.016467,ZTF21aasgnqa,150.838518,38.408163,9.299792,59308.264919,stamp_classifier,SN,0.471552,1
9,source_1,160.183014,33.016467,ZTF21aasuggn,166.212719,31.622886,5.281786,59308.227685,stamp_classifier,SN,0.430196,1


# Open the resulting objects in the ALeRCE explorer

In [27]:
url = "https://alerce.online/?" + "&".join(["oid=%s" % oid for oid in matches.oid.values]) + "&count=true&page=1&perPage=1000&sortDesc=false"
print(url)

https://alerce.online/?oid=ZTF21aasvhwh&oid=ZTF21aaroydq&oid=ZTF21aarnbox&oid=ZTF21aaqyqwk&oid=ZTF21aastzsn&oid=ZTF21aaqzdut&oid=ZTF21aardzpu&oid=ZTF21aarowtc&oid=ZTF21aasgnqa&oid=ZTF21aasuggn&oid=ZTF21aappwvm&oid=ZTF21aarngcx&oid=ZTF21aaqyraq&oid=ZTF21aardzhu&oid=ZTF21aardzlu&oid=ZTF21aarxuih&oid=ZTF21aarebbl&oid=ZTF21aarnlkz&oid=ZTF21aasyydo&oid=ZTF21aaszaoy&oid=ZTF21aareduf&oid=ZTF21aarovmb&oid=ZTF21aargpjl&oid=ZTF21aarohkc&oid=ZTF21aarohyu&oid=ZTF21aarwlht&oid=ZTF21aardszu&oid=ZTF21aarxsfm&oid=ZTF21aardzhu&count=true&page=1&perPage=1000&sortDesc=false
